<a href="https://colab.research.google.com/github/Haaz123/Absenteeism_Prediction/blob/master/Absenteeism_preprocessing_data%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing

In [2]:
raw_unprocessed_data1 = pd.read_csv('1.2 Absenteeism_data.csv.csv')
df = raw_unprocessed_data1.copy()

In [3]:
# remove "#" to view whole data 
#pd.options.display.max_columns= None
#pd.options.display.max_rows= None

In [4]:
#df.head()

In [5]:
df = df.drop(['ID'], axis= 1) # drop ID as it has no meaning to our model

In [6]:
#df.head()
  
    

In [7]:
reasons_columns = pd.get_dummies(df['Reason for Absence'], drop_first=True) # reason is categorical so creating it's columns


In [8]:
reasons_columns.columns.values

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 21, 22, 23, 24, 25, 26, 27, 28])

In [9]:
df = df.drop(['Reason for Absence'], axis= 1)

In [10]:
# Now lets divide reaasons of absence into 4 groups intead of 28 

reason_type1 = reasons_columns.loc[:, 1:14].max(axis=1)
reason_type2 = reasons_columns.loc[:, 15:17].max(axis=1)
reason_type3 = reasons_columns.loc[:, 18:21].max(axis=1)
reason_type4 = reasons_columns.loc[:, 22:].max(axis=1)

In [11]:
df = pd.concat([ reason_type1, reason_type2, reason_type3, reason_type4, df], axis = 1)


In [12]:
df['Education']= df['Education'].map({1:0, 2:1, 3:1, 4:1})

In [13]:
df.head()

,0,1,2,3,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,07/07/2015,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,14/07/2015,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,15/07/2015,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,16/07/2015,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,23/07/2015,289,36,33,239.554,30,0,2,1,2


In [14]:
df.columns.values

array([0, 1, 2, 3, 'Date', 'Transportation Expense', 'Distance to Work',
       'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours'], dtype=object)

In [15]:
columns_names = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours']

In [16]:
df.columns = columns_names

In [17]:
#df.head()

In [18]:
df_reason_mod = df.copy() # creating a check point


In [19]:
# now lets focus on Date column

df_reason_mod['Date'] = pd.to_datetime(df_reason_mod['Date'], format='%d/%m/%Y')

In [20]:
#df_reason_mod['Date']

In [21]:
df_reason_mod['Date'][180].weekday()

3

In [22]:
month_list = [] 
#year_list = []
day_list = []



In [23]:
for i in range(700): 
  month_list.append(df_reason_mod['Date'][i].month)
  day_list.append(df_reason_mod['Date'][i].weekday())
  #year_list.append(df_reason_mod['Date'][i].year)
  
  

In [24]:
df_reason_mod['Day of the Week'] = day_list 



In [25]:
df_reason_mod['Month of Year'] = month_list

In [26]:
#df_reason_mod.head()


In [27]:
df_reason_mod = df_reason_mod.drop(['Date'], axis= 1) # now lets drop date section as we do not need it

In [28]:
#df_reason_mod.columns.values

In [29]:
df_reason_mod = df_reason_mod[['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 
       'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Day of the Week',
       'Month of Year', 'Absenteeism Time in Hours']]

In [30]:
#df_reason_mod.head()

In [31]:
# lets have targets in 2 clusters 0 (if absent for less than 3 hours) and 1 ( if absent for more than 3H)
targets = np.where(df_reason_mod['Absenteeism Time in Hours']> df_reason_mod['Absenteeism Time in Hours'].median(), 1,0)

In [32]:
#targets

In [33]:
# drop the absenteeisn column
df_with_targets = df_reason_mod.drop(['Absenteeism Time in Hours'] , axis =1)

In [34]:
#df_with_targets.head()

In [35]:
# set a new colunm with target values
df_with_targets['Absenteeism Time Targets'] = targets

In [36]:
#df_with_targets.head()

In [37]:
#save the data for further processing 
df_with_targets.to_csv('Absenteeism_Preprocessed.csv', index = False)